In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml # If you don't have this package use 'pip install pyyaml' into commandline or terminal
# Matplotlib Config
%matplotlib inline
plt.style.use('fivethirtyeight')
# Turn 'building.yaml' file into a python dictionary using PyYAML 
with open('building.yaml') as f:
    building_data = f.read()
    building_data = yaml.load(building_data)
    f.close()
building = pd.DataFrame(building_data) #building contains only two rows out of which only one(first one) contains actual info
building = pd.DataFrame(building['building1'][0])
    
### Column Names for our dataframe
col_names = ['sec', 
             'agent_id',
             'agent_type',
             'has_luggage',
             'is_disabled',
             'X', 
             'Y',
             'Z',
             'velocity',
             'queue_id',
             'lookUp_X',
             'lookUp_Y']
df1 = pd.read_csv('Agents/thread_0.txt', header=0, names=col_names)
df2 = pd.read_csv('Agents/thread_1.txt', header=0, names=col_names)
df3 = pd.read_csv('Agents/thread_2.txt', header=0, names=col_names)
df4 = pd.read_csv('Agents/thread_3.txt', header=0, names=col_names)
df5 = pd.read_csv('Agents/thread_4.txt', header=0, names=col_names)
df6 = pd.read_csv('Agents/thread_5.txt', header=0, names=col_names)
df7 = pd.read_csv('Agents/thread_6.txt', header=0, names=col_names)
df8 = pd.read_csv('Agents/thread_7.txt', header=0, names=col_names)

frames = [df1,df2,df3,df4,df5,df6,df7,df8]
df = pd.concat(frames)

In [140]:
#list of all x,y and z points in the building
x_list = []
y_list = []
z_list = []
for i in building.iterrows():
    if i[1]['collision_geometry']['type'] == 'polygon':
        coordinates = i[1]['collision_geometry']['triangles']
    if i[1]['collision_geometry']['type'] == 'box':
        coordinates = i[1]['config']['position']
    x_list += coordinates[::3]
    y_list += coordinates[1::3]
    z_list += coordinates[2::3] 
#there are 33584 x,y,z coordinates from the building file

In [141]:
max_point = (max(x_list),max(y_list),max(z_list))
min_point = (min(x_list),min(y_list),min(z_list))
(max_point[0] - min_point[0],max_point[1] - min_point[1],max_point[2] - min_point[2])

(281.15, 153.35, 25.5)

In [142]:
agents = df[df['Z']>=0] #taking only the points which have positive Z value
agents.shape

(3830209, 12)

In [143]:
agents.head()

,sec,agent_id,agent_type,has_luggage,is_disabled,X,Y,Z,velocity,queue_id,lookUp_X,lookUp_Y
0,70201,0,4,0.0,0.0,105.955,20.712,7.0,0.586,-1.0,105.500,22.130
1,70201,8,4,0.0,0.0,82.995,14.989,1.0,0.720,-1.0,83.495,18.265
2,70201,16,1,0.0,0.0,49.066,-72.638,1.0,0.737,-1.0,49.822,-72.954
3,70201,24,1,0.0,0.0,111.617,28.309,10.0,2.944,-1.0,108.060,30.175
4,70201,32,1,0.0,0.0,108.920,34.867,4.0,1.386,-1.0,103.980,32.855


In [144]:
xlimt = (min_point[0],max_point[0])
ylimt = (min_point[1],max_point[1])
zlimt = (min_point[2],max_point[2])
print(xlimt,ylimt,zlimt)

(-139.15, 142.0) (-78.41, 74.94) (0.0, 25.5)


In [155]:
import unittest
'''class btree:
    def __init__(self):
        self.root = None
        self.size = 0
    def put(self,agents,xlim,ylim,zlim,parent=None,left=None,right=None):
        if self.root:
            pass
        else:
            self.root = TreeNode(agents,xlim,ylim,zlim)
        self.increase_ct()
        return TreeNode(agents,xlim,ylim,zlim)
    
    def increase_ct(self):
        self.size += 1'''
        
class TreeNode:
    def __init__(self,agents,xlim,ylim,zlim,parent=None,left=None,right=None): #agents is a pandas df
        self.xlim = xlim
        self.ylim = ylim
        self.zlim = zlim #each xlim,ylim and zlim is a tuple denoting limits on each axis
        self.count = self.get_count(agents) #count gets the number of agents for each node
        self.traffic_density = self.get_density()
        self.leftChild = left
        self.rightChild = right
        self.parent = parent
        
    def get_count(self,agents): #gets the number of agents within the x,y and z limits
        c = agents[(agents['X']>=self.xlim[0]) & (agents['X']<self.xlim[1]) & (agents['Y']>=self.ylim[0]) & (agents['Y']<self.ylim[1]) & (agents['Z']>=self.zlim[0]) & (agents['Z']<self.zlim[1])].shape[0]
        return c
    
    def get_density(self):  #calculates the space usage per sq. area
        x = self.xlim[1]-self.xlim[0] #using only x and y axis for area calculation
        y = self.ylim[1]-self.ylim[0]
        area = x*y
        return round(self.count/area,2)
    
    def split(self,agents,axis): #axis should be either 'x' or 'y' or 'z'
        if axis == 'x':
            mp = round((self.xlim[0]+self.xlim[1])/2,1)  #mp is mid point
            l_xlim = (self.xlim[0],mp)   #l_xlim is lower_x_limit
            u_xlim = (mp,self.xlim[1])   #u_xlim is upper_x_limit
            self.leftChild = TreeNode(agents,l_xlim,self.ylim,self.zlim,parent = self)
            self.rightChild = TreeNode(agents,u_xlim,self.ylim,self.zlim,parent = self)
        elif axis == 'y':
            mp = round((self.ylim[0]+self.ylim[1])/2,1)
            l_ylim = (self.ylim[0],mp)
            u_ylim = (mp,self.ylim[1])
            self.leftChild = TreeNode(agents,self.xlim,l_ylim,self.zlim,parent = self)
            self.rightChild = TreeNode(agents,self.xlim,u_ylim,self.zlim,parent = self)
        elif axis == 'z':
            mp = round((self.zlim[0]+self.zlim[1])/2,1)
            l_zlim = (self.zlim[0],mp)
            u_zlim = (mp,self.zlim[1])
            self.leftChild = TreeNode(agents,self.xlim,self.ylim,l_zlim,parent = self)
            self.rightChild = TreeNode(agents,self.xlim,self.ylim,u_zlim,parent = self)
        else:
            pass
        return (self.leftChild,self.rightChild)
    
    def __str__(self):
        s = "Agent Traffic in node : " + str(self.count)
        s += "\n" + "X : " + str(self.xlim) + "\n" + "Y : " + str(self.ylim) + "\n" + "Z : " + str(self.zlim) + "\n" + "Traffic Density : " + str(self.traffic_density) + " usage/sq.area" + "\n" 
        return s

In [156]:
#root initialization
root = TreeNode(agents,xlimt,ylimt,zlimt)  #pass the whole building limits and whole set of agents data into root
print(root)

Agent Traffic in node : 3830203
X : (-139.15, 142.0)
Y : (-78.41, 74.94)
Z : (0.0, 25.5)
Traffic Density : 88.84 usage/sq.area



In [147]:
#recursive code to split building spatially
def building_split(node):
    if node.count < 2000: #this parameter can control the depth of the tree
        return node
    if node.zlim[1]-node.zlim[0] > 2: #check for this bcoz after a few levels z split wont be needed
        for i in node.split(agents,'z'):
            for j in i.split(agents,'x'):
                for k in j.split(agents,'y'):
                    building_split(k) #recursion call
    elif node.xlim[1]-node.xlim[0] > 20:
        for j in node.split(agents,'x'):
            for k in j.split(agents,'y'):
                building_split(k) #recursion call
    return node             

In [157]:
#function call - runs for a looong time - about 10 minutes
building_split(root)

In [158]:
#function to get all the nodes at a certain depth from root
def get_nodes_at_depth(root,depth):
    node_list = [root]
    child_list = []
    for i in range(depth):
        for j in node_list:
            child_list.append(j.leftChild)
            child_list.append(j.rightChild)
        node_list = child_list
        child_list = []
    return node_list
#check - the sum of all counts from any depth should add upto the total number of counts in root

In [159]:
nodes = get_nodes_at_depth(root,3)  #change depth and call for different depths
print("The number of nodes at this level is " + str(len(nodes)) + "\n")
for i in nodes:
    print(i)

The number of nodes at this level is 8

Agent Traffic in node : 647228
X : (-139.15, 1.4)
Y : (-78.41, -1.7)
Z : (0.0, 12.8)
Traffic Density : 60.03 usage/sq.area

Agent Traffic in node : 0
X : (-139.15, 1.4)
Y : (-1.7, 74.94)
Z : (0.0, 12.8)
Traffic Density : 0.0 usage/sq.area

Agent Traffic in node : 1540471
X : (1.4, 142.0)
Y : (-78.41, -1.7)
Z : (0.0, 12.8)
Traffic Density : 142.83 usage/sq.area

Agent Traffic in node : 921111
X : (1.4, 142.0)
Y : (-1.7, 74.94)
Z : (0.0, 12.8)
Traffic Density : 85.48 usage/sq.area

Agent Traffic in node : 216488
X : (-139.15, 1.4)
Y : (-78.41, -1.7)
Z : (12.8, 25.5)
Traffic Density : 20.08 usage/sq.area

Agent Traffic in node : 0
X : (-139.15, 1.4)
Y : (-1.7, 74.94)
Z : (12.8, 25.5)
Traffic Density : 0.0 usage/sq.area

Agent Traffic in node : 152084
X : (1.4, 142.0)
Y : (-78.41, -1.7)
Z : (12.8, 25.5)
Traffic Density : 14.1 usage/sq.area

Agent Traffic in node : 352821
X : (1.4, 142.0)
Y : (-1.7, 74.94)
Z : (12.8, 25.5)
Traffic Density : 32.74 usag